In [6]:
import numpy as np
import pandas as pd

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
#import torch.nn as nn

In [7]:
BATCH_SIZE = 32
NUM_EPOCHS = 100

In [2]:
train_dataset = pd.read_csv("/content/train.csv")
test_dataset = pd.read_csv("/content/test.csv")

In [3]:
train_dataset.head()

,author,lung,council,solution,quite,rain,hair,skill,difficulty,add,...,stocking,near,oil,dive,many,run,tender,asleep,eat,sweep
0,Mason Reed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ava Thompson,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Liam Carter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mason Reed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Olivia Bennett,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data = np.array(train_dataset)
Y = data[:, 0]
print(Y)
le = LabelEncoder()
Y_numerical = le.fit_transform(Y)
print(Y_numerical)
print(le.classes_)

['Mason Reed' 'Ava Thompson' 'Liam Carter' ... 'Ava Thompson'
 'Ethan Brooks' 'Mason Reed']
[3 0 2 ... 0 1 3]
['Ava Thompson' 'Ethan Brooks' 'Liam Carter' 'Mason Reed' 'Olivia Bennett']


In [4]:
test_dataset.head()

,lung,council,solution,quite,rain,hair,skill,difficulty,add,pull,...,stocking,near,oil,dive,many,run,tender,asleep,eat,sweep
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
class AuthorClassifier(torch.nn.Module):
  def __init__(self):
    super(AuthorClassifier, self).__init__()

    # linear layers (B, 380) -> (B, 5)
    self.linear_layers = torch.nn.ModuleList()
    self.linear_layers.append(torch.nn.Linear(380, 256))
    self.linear_layers.append(torch.nn.Linear(256, 128))
    self.linear_layers.append(torch.nn.Linear(128, 16))
    self.linear_layers.append(torch.nn.Linear(16, 5))

  def forward(self, x):
    for layer in self.linear_layers:
      x = layer(x)

    return x


In [49]:
class TextDataset(torch.utils.data.Dataset):
  def __init__(self, file_addr):
    super(TextDataset, self).__init__()

    data = np.array(pd.read_csv(file_addr))
    self.X = data[:, 1:].astype(np.float32)
    self.Y = data[:, 0]
    self.le = LabelEncoder()
    self.Y_numerical = self.le.fit_transform(self.Y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    sample_X = self.X[index]
    sample_Y = self.Y_numerical[index]

    return sample_X, sample_Y


In [15]:
ath_clf = AuthorClassifier()

random_input = torch.randn((16, 380))

output = ath_clf(random_input)
print(output.shape)


torch.Size([16, 5])


In [32]:
ath_dataset = TextDataset("/content/train.csv")
ath_dataloder = torch.utils.data.DataLoader(dataset=ath_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [33]:
for x, label in ath_dataloder:
  print(x.shape)
  print(label)
  break

torch.Size([32, 380])
tensor([3, 0, 2, 4, 4, 2, 3, 0, 1, 1, 1, 3, 0, 1, 1, 4, 3, 1, 2, 1, 2, 1, 4, 3,
        4, 1, 4, 0, 4, 3, 0, 0])


In [38]:
predictions = ath_clf(x)
_, indices = torch.max(predictions, axis=1)
print(indices)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


In [40]:
ath_dataset = TextDataset("/content/train.csv")
ath_dataloder = torch.utils.data.DataLoader(dataset=ath_dataset, batch_size=BATCH_SIZE, shuffle=True)

ath_clf = AuthorClassifier()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ath_clf.parameters(), lr=0.001)

In [41]:
for epoch in range(NUM_EPOCHS):
  list_of_losses = list()
  for x, y in ath_dataloder:
    ath_clf.zero_grad()

    predictions = ath_clf(x)

    loss = criterion(predictions, y)
    loss.backward()
    optimizer.step()

    list_of_losses.append(loss.item())

  print(f"Epoch: {epoch+1}, Avg loss: {np.mean(list_of_losses)}")


Epoch: 1, Avg loss: 0.9020199227333069
Epoch: 2, Avg loss: 0.5310618034005166
Epoch: 3, Avg loss: 0.48014617040753366
Epoch: 4, Avg loss: 0.4651152931153774
Epoch: 5, Avg loss: 0.4502976153790951
Epoch: 6, Avg loss: 0.4402751947939396
Epoch: 7, Avg loss: 0.4329556003212929
Epoch: 8, Avg loss: 0.4282388833165169
Epoch: 9, Avg loss: 0.41835219457745554
Epoch: 10, Avg loss: 0.41916834488511084
Epoch: 11, Avg loss: 0.41680546224117276
Epoch: 12, Avg loss: 0.4183227360248566
Epoch: 13, Avg loss: 0.40735626354813576
Epoch: 14, Avg loss: 0.4105776792019606
Epoch: 15, Avg loss: 0.41098604768514635
Epoch: 16, Avg loss: 0.40555464848876
Epoch: 17, Avg loss: 0.4034690663218498
Epoch: 18, Avg loss: 0.4000151006877422
Epoch: 19, Avg loss: 0.4042209202051163
Epoch: 20, Avg loss: 0.40381007790565493
Epoch: 21, Avg loss: 0.4063101524859667
Epoch: 22, Avg loss: 0.40459278419613837
Epoch: 23, Avg loss: 0.39617191553115844
Epoch: 24, Avg loss: 0.3952113954722881
Epoch: 25, Avg loss: 0.3958150053024292
Ep

In [43]:
predictions = ath_clf(x)
_, indices = torch.max(predictions, axis=1)
print(indices)
print(label)

tensor([1, 4, 2, 0, 0, 1, 0, 2, 1, 4, 3, 1, 3, 2, 2, 3, 1, 4, 4, 3, 3, 2, 2, 2,
        3, 0, 2, 4])
tensor([3, 0, 2, 4, 4, 2, 3, 0, 1, 1, 1, 3, 0, 1, 1, 4, 3, 1, 2, 1, 2, 1, 4, 3,
        4, 1, 4, 0, 4, 3, 0, 0])


In [52]:
list_of_labels = list()
list_of_predictions = list()

with torch.no_grad():
  for x, y in ath_dataloder:

    _, predictions = torch.max(ath_clf(x), axis=1)

    list_of_labels.extend(y.numpy())
    list_of_predictions.extend(predictions.numpy())


In [53]:
print(list_of_labels)
print(list_of_predictions)

[2, 0, 3, 2, 0, 3, 4, 3, 2, 4, 0, 1, 1, 3, 2, 0, 4, 4, 0, 3, 2, 1, 4, 1, 2, 0, 2, 4, 0, 4, 4, 0, 4, 3, 1, 0, 4, 3, 3, 1, 0, 0, 1, 3, 1, 1, 0, 3, 3, 2, 3, 1, 0, 4, 4, 4, 0, 2, 3, 3, 4, 0, 2, 1, 2, 3, 4, 1, 2, 0, 4, 0, 3, 4, 1, 0, 2, 2, 0, 4, 3, 0, 4, 2, 1, 2, 4, 3, 0, 1, 1, 4, 1, 0, 2, 4, 0, 2, 0, 3, 0, 4, 3, 4, 1, 1, 3, 0, 1, 2, 0, 3, 1, 1, 1, 3, 2, 3, 0, 2, 4, 2, 1, 2, 1, 1, 3, 0, 0, 0, 4, 3, 0, 2, 3, 3, 1, 2, 3, 2, 3, 1, 0, 1, 2, 2, 4, 0, 4, 1, 2, 0, 3, 2, 4, 3, 2, 4, 0, 1, 1, 3, 1, 3, 4, 0, 3, 2, 2, 2, 1, 2, 0, 2, 3, 3, 3, 1, 2, 1, 3, 0, 1, 3, 3, 1, 2, 1, 1, 3, 1, 1, 1, 1, 0, 4, 2, 3, 1, 1, 4, 4, 1, 4, 4, 4, 3, 1, 0, 3, 0, 1, 2, 1, 0, 3, 0, 0, 1, 2, 3, 4, 3, 3, 3, 4, 3, 4, 3, 1, 3, 1, 1, 4, 1, 3, 0, 1, 0, 4, 1, 4, 3, 4, 4, 3, 3, 0, 0, 4, 1, 1, 4, 2, 3, 2, 0, 3, 2, 4, 0, 1, 1, 1, 0, 2, 4, 0, 3, 2, 3, 3, 0, 4, 0, 0, 3, 0, 4, 4, 4, 2, 0, 1, 0, 0, 2, 4, 2, 4, 2, 3, 1, 2, 1, 3, 0, 4, 2, 1, 4, 0, 4, 2, 2, 2, 2, 2, 1, 4, 3, 3, 0, 2, 4, 3, 0, 2, 1, 1, 0, 0, 1, 3, 1, 4, 1, 4, 1, 3, 3, 1, 0, 

In [55]:
print(accuracy_score(list_of_labels, list_of_predictions))

0.8764080100125157


# Train-Test Split

In [10]:
from sklearn.model_selection import train_test_split

In [9]:
class TextDataset(torch.utils.data.Dataset):
  def __init__(self, data):
    super(TextDataset, self).__init__()

    data = np.array(data)
    self.X = data[:, 1:].astype(np.float32)
    self.Y = data[:, 0]
    self.le = LabelEncoder()
    self.Y_numerical = self.le.fit_transform(self.Y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    sample_X = self.X[index]
    sample_Y = self.Y_numerical[index]

    return sample_X, sample_Y


In [100]:
train_dataset = pd.read_csv("/content/train.csv")

In [59]:
train_data, test_data = train_test_split(train_dataset, test_size=0.2, shuffle=True)

In [62]:
train_data.head()
print(len(train_data))

2556


In [61]:
test_data.head()
print(len(test_data))

640


In [79]:
ath_train_dataset = TextDataset(train_data)
ath_train_dataloder = torch.utils.data.DataLoader(dataset=ath_train_dataset, batch_size=BATCH_SIZE, shuffle=True)

ath_test_dataset = TextDataset(test_data)
ath_test_dataloder = torch.utils.data.DataLoader(dataset=ath_test_dataset, batch_size=BATCH_SIZE, shuffle=True)

ath_clf = AuthorClassifier()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ath_clf.parameters(), lr=0.0001)

In [83]:
for epoch in range(NUM_EPOCHS):
  list_of_losses = list()
  for x, y in ath_train_dataloder:
    ath_clf.zero_grad()

    predictions = ath_clf(x)

    loss = criterion(predictions, y)
    loss.backward()
    optimizer.step()

    list_of_losses.append(loss.item())

  print(f"Epoch: {epoch+1}, Avg loss: {np.mean(list_of_losses)}")


Epoch: 1, Avg loss: 0.31940041286870835
Epoch: 2, Avg loss: 0.31784738288260994
Epoch: 3, Avg loss: 0.3182787746191025
Epoch: 4, Avg loss: 0.31808258816599844
Epoch: 5, Avg loss: 0.3168470785021782
Epoch: 6, Avg loss: 0.3186693095602095
Epoch: 7, Avg loss: 0.3166503610089421
Epoch: 8, Avg loss: 0.31723378589376805
Epoch: 9, Avg loss: 0.31560068065300584
Epoch: 10, Avg loss: 0.3163878694176674
Epoch: 11, Avg loss: 0.31637666281312704
Epoch: 12, Avg loss: 0.3151310818269849
Epoch: 13, Avg loss: 0.31655296916142106
Epoch: 14, Avg loss: 0.31540687223896385
Epoch: 15, Avg loss: 0.31515681445598603
Epoch: 16, Avg loss: 0.31426525861024857
Epoch: 17, Avg loss: 0.3144592451862991
Epoch: 18, Avg loss: 0.31418724423274397
Epoch: 19, Avg loss: 0.3145308017730713
Epoch: 20, Avg loss: 0.31335698133334516
Epoch: 21, Avg loss: 0.31416023736819626
Epoch: 22, Avg loss: 0.3134372979402542
Epoch: 23, Avg loss: 0.31262184157967565
Epoch: 24, Avg loss: 0.31269003357738256
Epoch: 25, Avg loss: 0.31284332834

In [86]:
list_of_labels = list()
list_of_predictions = list()

with torch.no_grad():
  for x, y in ath_test_dataloder:

    _, predictions = torch.max(ath_clf(x), axis=1)

    list_of_labels.extend(y.numpy())
    list_of_predictions.extend(predictions.numpy())


In [87]:
print(accuracy_score(list_of_labels, list_of_predictions))

0.771875


## KFold

In [11]:
from sklearn.model_selection import KFold
train_dataset = pd.read_csv("/content/train.csv")

In [19]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

In [39]:
class AuthorClassifier(torch.nn.Module):
  def __init__(self):
    super(AuthorClassifier, self).__init__()

    # linear layers (B, 380) -> (B, 5)
    self.linear_layers = torch.nn.ModuleList()
    self.linear_layers.append(torch.nn.Linear(380, 256))
    self.linear_layers.append(torch.nn.Linear(256, 128))
    self.linear_layers.append(torch.nn.Dropout(0.2)) # torch.nn.LeakyReLU(0.05)
    self.linear_layers.append(torch.nn.Linear(128, 16))
    self.linear_layers.append(torch.nn.Dropout(0.2))
    self.linear_layers.append(torch.nn.Linear(16, 5))
    self.linear_layers.append(torch.nn.Softmax())

  def forward(self, x):
    for layer in self.linear_layers:
      x = layer(x)

    return x


In [40]:
def train_model(model, dataloader, optimizer, criterion):
  model.train()
  for epoch in range(NUM_EPOCHS):
    for x, y in dataloader:
      x, y = x.to(device), y.to(device)
      model.zero_grad()

      predictions = model(x)

      loss = criterion(predictions, y)
      loss.backward()
      optimizer.step()


  return model

def test_model(model, dataloader):
  list_of_labels = list()
  list_of_predictions = list()
  model.eval()

  with torch.no_grad():
    for x, y in dataloader:
      x, y = x.to(device), y.to(device)
      _, predictions = torch.max(model(x), axis=1)

      list_of_labels.extend(y.detach().cpu().numpy())
      list_of_predictions.extend(predictions.detach().cpu().numpy())

  return accuracy_score(list_of_labels, list_of_predictions)



In [ ]:
kfold_dataset = KFold(n_splits=5, shuffle=True)
list_of_accuracies = list()
for train_index, test_index in kfold_dataset.split(train_dataset):


  ath_clf = AuthorClassifier().to(device)

  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(ath_clf.parameters(), lr=0.001)

  ath_train_dataset = TextDataset(train_dataset.iloc[train_index])
  ath_train_dataloder = torch.utils.data.DataLoader(dataset=ath_train_dataset, batch_size=BATCH_SIZE, shuffle=True)

  ath_test_dataset = TextDataset(train_dataset.iloc[test_index])
  ath_test_dataloder = torch.utils.data.DataLoader(dataset=ath_test_dataset, batch_size=BATCH_SIZE, shuffle=True)

  train_model(ath_clf, ath_train_dataloder, optimizer, criterion)
  list_of_accuracies.append(test_model(ath_clf, ath_test_dataloder))
  print(list_of_accuracies[-1])


In [42]:
print(f"Avg acc: {np.mean(list_of_accuracies)}, std: {np.std(list_of_accuracies)}")

Avg acc: 0.7919268388106417, std: 0.00846942027748635


In [32]:
print(f"Avg acc: {np.mean(list_of_accuracies)}, std: {np.std(list_of_accuracies)}")

Avg acc: 0.8000621087636933, std: 0.012064276373240326


## Final Model

In [64]:
ath_dataset = TextDataset("/content/train.csv")
ath_dataloder = torch.utils.data.DataLoader(dataset=ath_dataset, batch_size=BATCH_SIZE, shuffle=True)

ath_clf = AuthorClassifier().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ath_clf.parameters(), lr=0.001)

In [ ]:
for epoch in range(NUM_EPOCHS):
  list_of_losses = list()
  for x, y in ath_dataloder:
    ath_clf.zero_grad()
    x, y = x.to(device), y.to(device)
    predictions = ath_clf(x)

    loss = criterion(predictions, y)
    loss.backward()
    optimizer.step()

    list_of_losses.append(loss.item())

  print(f"Epoch: {epoch+1}, Avg loss: {np.mean(list_of_losses)}")


In [67]:
test_dataframe = pd.read_csv("/content/test.csv")

In [44]:
test_dataframe.head()

,lung,council,solution,quite,rain,hair,skill,difficulty,add,pull,...,stocking,near,oil,dive,many,run,tender,asleep,eat,sweep
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
class TestTextDataset(torch.utils.data.Dataset):
  def __init__(self, data):
    super(TestTextDataset, self).__init__()

    data = np.array(data)
    self.X = data[:, 0:].astype(np.float32)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    sample_X = self.X[index]

    return sample_X


In [69]:
test_dataset = TestTextDataset(test_dataframe)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [70]:
list_of_predictions = list()
ath_clf.eval()

with torch.no_grad():
  for x in test_dataloader:
    x = x.to(device)
    _, predictions = torch.max(ath_clf(x), axis=1)

    list_of_predictions.extend(predictions.detach().cpu().numpy())

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [71]:
print(list_of_predictions)

[4, 1, 2, 2, 4, 4, 0, 2, 2, 0, 2, 2, 3, 0, 2, 3, 0, 3, 4, 0, 2, 4, 0, 1, 4, 2, 2, 4, 2, 3, 0, 3, 4, 4, 0, 1, 3, 2, 2, 4, 2, 2, 4, 3, 2, 1, 3, 0, 0, 4, 0, 2, 2, 4, 3, 3, 2, 0, 0, 0, 1, 2, 2, 3, 0, 1, 1, 2, 0, 4, 2, 4, 3, 4, 4, 2, 2, 2, 1, 0, 0, 3, 2, 2, 4, 2, 1, 2, 2, 1, 3, 4, 1, 4, 4, 3, 0, 1, 2, 2, 1, 0, 1, 2, 4, 2, 1, 1, 2, 3, 4, 0, 4, 1, 3, 0, 0, 3, 2, 2, 4, 4, 4, 3, 2, 4, 2, 1, 1, 0, 4, 0, 2, 2, 4, 3, 1, 3, 2, 1, 1, 1, 0, 2, 0, 0, 3, 3, 0, 4, 0, 1, 0, 3, 4, 4, 4, 0, 3, 3, 2, 1, 4, 0, 3, 3, 4, 0, 0, 0, 4, 3, 2, 2, 4, 0, 0, 2, 4, 3, 2, 0, 0, 3, 1, 2, 1, 1, 2, 4, 4, 3, 3, 4, 3, 2, 4, 1, 2, 3, 3, 1, 1, 1, 3, 4, 0, 4, 3, 2, 1, 0, 1, 2, 2, 4, 2, 0, 4, 2, 1, 3, 3, 3, 1, 1, 4, 0, 2, 4, 1, 3, 3, 4, 1, 3, 2, 3, 0, 4, 0, 0, 2, 1, 4, 4, 2, 0, 3, 2, 1, 3, 4, 1, 4, 1, 3, 3, 0, 2, 0, 0, 4, 2, 1, 4, 1, 3, 1, 3, 1, 4, 3, 1, 3, 1, 2, 3, 2, 3, 3, 4, 3, 1, 0, 2, 3, 0, 4, 4, 0, 0, 2, 3, 2, 0, 4, 3, 3, 3, 0, 2, 1, 3, 2, 4, 1, 1, 2, 1, 3, 0, 2, 1, 1, 2, 3, 2, 1, 4, 2, 3, 0, 0, 4, 4, 2, 3, 1, 0, 4, 0, 1, 

In [72]:
print(ath_dataset.le.inverse_transform(list_of_predictions))

['Olivia Bennett' 'Ethan Brooks' 'Liam Carter' 'Liam Carter'
 'Olivia Bennett' 'Olivia Bennett' 'Ava Thompson' 'Liam Carter'
 'Liam Carter' 'Ava Thompson' 'Liam Carter' 'Liam Carter' 'Mason Reed'
 'Ava Thompson' 'Liam Carter' 'Mason Reed' 'Ava Thompson' 'Mason Reed'
 'Olivia Bennett' 'Ava Thompson' 'Liam Carter' 'Olivia Bennett'
 'Ava Thompson' 'Ethan Brooks' 'Olivia Bennett' 'Liam Carter'
 'Liam Carter' 'Olivia Bennett' 'Liam Carter' 'Mason Reed' 'Ava Thompson'
 'Mason Reed' 'Olivia Bennett' 'Olivia Bennett' 'Ava Thompson'
 'Ethan Brooks' 'Mason Reed' 'Liam Carter' 'Liam Carter' 'Olivia Bennett'
 'Liam Carter' 'Liam Carter' 'Olivia Bennett' 'Mason Reed' 'Liam Carter'
 'Ethan Brooks' 'Mason Reed' 'Ava Thompson' 'Ava Thompson'
 'Olivia Bennett' 'Ava Thompson' 'Liam Carter' 'Liam Carter'
 'Olivia Bennett' 'Mason Reed' 'Mason Reed' 'Liam Carter' 'Ava Thompson'
 'Ava Thompson' 'Ava Thompson' 'Ethan Brooks' 'Liam Carter' 'Liam Carter'
 'Mason Reed' 'Ava Thompson' 'Ethan Brooks' 'Ethan Brook

### Dynamic Layers

In [80]:
class AuthorClassifier(torch.nn.Module):
  def __init__(self, input_features, neurons_in_layers):
    super(AuthorClassifier, self).__init__()

    # linear layers (B, 380) -> (B, 5)
    self.linear_layers = torch.nn.ModuleList()

    for layer_index, neuron in enumerate(neurons_in_layers):
      self.linear_layers.append(torch.nn.Linear(input_features, neuron))
      if layer_index >= 2:
        self.linear_layers.append(torch.nn.LeakyReLU(0.05))
      input_features = neuron

    self.linear_layers.append(torch.nn.Softmax())

  def forward(self, x):
    for layer in self.linear_layers:
      x = layer(x)

    return x

In [81]:
clf = AuthorClassifier(input_features=400, neurons_in_layers=[5, 10, 20, 100, 50])

random_input = torch.randn((10, 400))
print(clf(random_input).shape)

torch.Size([10, 50])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
